# 全链路最佳实践

以训练知乎bot为例，让AI变的不那么AI

内容包括：
- 环境搭建
- 数据集、dataset加载示例
- data-juicer处理脚本
- swift训练脚本
- evalscope评估脚本（客观指标 + human preference）

## 环境准备

```shell
conda create -n zhihu python=3.10
conda activate zhihu
```

安装modelscope、data-juicer、swift、evalscope

In [11]:
%pip install modelscope[framework]  # 模型库
%pip install py-data-juicer[sci]    # 数据处理库
%pip install ms-swift[llm]          # 训练库
%pip install ms-swift[eval]         # 评测库

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
  Using cached https://mirrors.cloud.aliyuncs.com/pypi/packages/21/68/c9ee59caec452530d32bb43104206ce1387d050adad05ee599616425ee7d/pyarrow-12.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.9 MB)
  Using cached https://mirrors.cloud.aliyuncs.com/pypi/packages/48/b7/6190994c06fb2cee1ff25c4326312a5c38c20e721424c332195f65af5190/datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached https://mirrors.cloud.aliyuncs.com/pypi/packages/0d/44/56c3f48d2bb83d76f5c970aef8e2c3ebd6a832f09e3621c5395371fe6999/typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached https://mirrors.cloud.aliyuncs.com/pypi/packages/ef/a6/080cace699e89a94bd4bf34e8c12821d1f05fe4d56a0742f797b231d9a40/pydantic-1.10.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: type

# ！！重启notebook环境！！
------

## 下载数据集

初步处理数据集，提取需要的字段，并处理成data-juicer需要的格式

In [2]:
from modelscope import MsDataset
from pprint import pprint

ds =  MsDataset.load('OmniData/Zhihu-KOL', cache_dir="data", split='train')
print(ds)
pprint(ds[0])

/mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA'],
    num_rows: 1006218
})
{'INSTRUCTION': '怎么说服男朋友买烤箱？',
 'METADATA': '{"question_id": 357137111.0, "answer_id": 914332816.0, "url": '
             '"https://www.zhihu.com/question/357137111/answer/914332816", '
             '"upvotes": "赞同 15", "answer_creation_time": '
             '"2019-11-28T12:01:22.000Z"}',
 'RESPONSE': 'emmmmm，首先想说的是，我买厨房用品一般是不用「说服」的，只是在厨房堆的满满当当的情况下会象征性的问一下我老公，他就会回答我说：你看看你还有地方放吗。然后我会思考一下，如果是特别想买的，就不会问他了。自己决定就好。 '
             '比如，前几天我又买了两个盘子~~~~他还不知道。 可以给题主看看我有多少的锅具：自家炒菜用什么锅好？各有什么优缺点？ '
             '说回烤箱的问题，买的时候处于热恋期，我告诉他我有一个买烤箱的计划。虽然他基本不吃点心，也不喜欢烘焙，但那个时期的他欣然同意并热情洋溢的给我选烤箱。可能是他有憧憬我会给他做什么好吃的吧。又因为我是一个不怎么吃甜食的湖南人，烤箱在我家烘焙的使用率很低。 '
             '但是！！你还是可以告诉他烤箱的作用是可以烤制各种肉类！！！我不相信有不喜欢吃肉的男生！！烤箱真的是可以烤一切的肉类，熟悉之后会觉得非常简单。 '
             '我很久以前用烤箱做的最多的就是烤羊排和烤鸡翅，我老公不怎么吃羊肉和鸡翅。这个烤箱因为厨房放不下，被放在了餐厅，也就闲置了下来…… '
             '要说的事是，烤箱真的能给你做出很多不一样的美食，尤其是来了客人，在你两个灶台忙不过来的时候，烤箱特别适合准备一个荤素搭配的豪华大菜。在烹饪其他需要爆炒的菜肴的空档去处理一下

/mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/datasets/builder.py:885: FutureWarning: 'try_from_hf_gcs' was deprecated in version 2.16.0 and will be removed in 3.0.0.
  warnings.warn(


In [3]:
# 处理 metadata
import json
# load json
metadata = list(map(lambda x: json.loads(x), ds['METADATA']))

# 处理 upvotes 
vote_list = []
for item in metadata:
    try:
        upvotes = item['upvotes'][3:]
        if not upvotes:
            votes = 0
        elif '万' in upvotes:
            votes = int(float(upvotes[:-2]) * 10000)
        else:
            votes = int(upvotes)
    except Exception as e:
        print(upvotes)
        votes = 0
    vote_list.append(votes)

In [5]:
# 写入 jsonl 文件
import pandas as pd

df = pd.DataFrame.from_dict({
    'query': ds['INSTRUCTION'],
    'response': ds['RESPONSE'],
    'upvotes': vote_list
})

print(len(df))

df.to_json("data/zhihu.jsonl", orient="records", lines=True, force_ascii=False)

1006218


## 使用data-juicer进行数据清洗

目的：获取高质量数据，用于模型训练

支持的算子：https://github.com/modelscope/data-juicer/blob/main/docs/Operators_ZH.md

全部算子的配置文件：https://github.com/modelscope/data-juicer/blob/main/configs/config_all.yaml

流程：
1. 编写yaml配置文件 （zhihu-bot.yaml）
2. 数据分析 `dj-analyze`
3. 筛选、过滤、去重 `dj-process`

简单配置原则：根据3sigma法则，若某个数据点超出均值±3σ的范围，通常被视为异常值。

### 数据分析

In [1]:
!dj-analyze --config zhihu-bot.yaml 

2024-08-19 11:09:10.991 | DEBUG    | data_juicer.utils.availability_utils:_is_package_available:116 - Detected torch version 2.4.0
2024-08-19 11:09:14.554 | INFO     | data_juicer:setup_mp:58 - Setting multiprocess start method to 'forkserver'.
2024-08-19 11:09:14.554 | DEBUG    | data_juicer:setup_cuda:72 - _USE_CUDA: True | MP: forkserver (MainProcess)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████████████████████████████████| 9.83M/9.83M [00:02<00:00, 4.57MB/s]
2024-08-19 11:09:29 | INFO     | data_juicer.config.config:531 - Back up the input config file [/mnt/data/data/user/maoyunlin.myl/tools/course/zhihu-bot.yaml] into the work_dir [/mnt/data/data/user/maoyunlin.myl/tools/course/data]
2024-08-19 11:09:29 | INFO     | data_juicer.config.config:552 - Configuration table: 
╒════════════════════════╤═════════════════════════════════════════════════════════════════

#### 数据集分析结果

- 箱型图
- 直方图
- 统计信息

在`data/analysis`路径下

|        |   alnum_ratio |   flagged_words_ratio | lang      |    lang_score |     perplexity |   special_char_ratio |         text_len |
|:-------|--------------:|----------------------:|:----------|--------------:|---------------:|---------------------:|-----------------:|
| count  |   1.00622e+06 |           1.00622e+06 | 1006218.0 |   1.00622e+06 |    1.00622e+06 |          1.00622e+06 |      1.00622e+06 |
| mean   |   0.871938    |           1.28188e-05 | nan       |   0.963631    | 2390           |          0.159879    |    717.802       |
| std    |   0.0793817   |           0.00120551  | nan       |   0.0976119   | 4733.66        |          0.0878637   |   1666.89        |
| min    |   0           |           0           | nan       |   0.0593122   |    0           |          0           |      1           |
| 25%    |   0.854922    |           0           | nan       |   0.976512    | 1500.4         |          0.118577    |     61           |
| 50%    |   0.883008    |           0           | nan       |   0.989479    | 2017.7         |          0.147059    |    236           |
| 75%    |   0.905219    |           0           | nan       |   0.994992    | 2695.5         |          0.183099    |    764           |
| max    |   1           |           0.6         | nan       |   1.00007     |    1.70447e+06 |          1           | 139406           |
| unique | nan           |         nan           | 99.0      | nan           |  nan           |        nan           |    nan           |
| top    | nan           |         nan           | zh        | nan           |  nan           |        nan           |    nan           |
| freq   | nan           |         nan           | 990697.0  | nan           |  nan           |        nan           |    nan           |

根据数据集特征，调整配置文件，进行数据处理

### 处理数据

注意：先进行筛选，再过滤，能减少数据处理的时间

In [3]:
!dj-process --config zhihu-bot.yaml 

2024-08-19 11:27:29.707 | DEBUG    | data_juicer.utils.availability_utils:_is_package_available:116 - Detected torch version 2.4.0
2024-08-19 11:27:33.479 | INFO     | data_juicer:setup_mp:58 - Setting multiprocess start method to 'forkserver'.
2024-08-19 11:27:33.479 | DEBUG    | data_juicer:setup_cuda:72 - _USE_CUDA: True | MP: forkserver (MainProcess)
2024-08-19 11:27:44 | INFO     | data_juicer.config.config:531 - Back up the input config file [/mnt/data/data/user/maoyunlin.myl/tools/course/zhihu-bot.yaml] into the work_dir [/mnt/data/data/user/maoyunlin.myl/tools/course/data]
2024-08-19 11:27:44 | INFO     | data_juicer.config.config:552 - Configuration table: 
╒════════════════════════╤════════════════════════════════════════════════════════════════════════════════════╕
│ key                    │ values                                                                             │
╞════════════════════════╪═══════════════════════════════════════════════════════════════════════════

### 划分训练集和测试集

In [5]:
import pandas as pd

data = pd.read_json("data/zhihu_refine.jsonl", lines=True)

def split_data(data, save=False, suffix=''):
    # split data into train and test, 9: 1
    train_data = data.sample(frac=0.9, random_state=42)
    test_data = data.drop(train_data.index)

    if suffix:
        suffix = '_' + suffix
    if save:
        train_data.to_json(f"data/zhihu_train{suffix}.jsonl", orient='records', lines=True, force_ascii=False)
        test_data.to_json(f"data/zhihu_test{suffix}.jsonl", orient='records', lines=True,  force_ascii=False)
    return train_data, test_data

train_data, test_data = split_data(data, save=True)

print(len(train_data))
print(len(test_data))

45000
5000


## 使用ms-swift训练模型

命令行参数：https://swift.readthedocs.io/zh-cn/latest/LLM/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html

(可选)安装 flash-attention 加快推理速度：

In [4]:
%pip install flash-attn --no-build-isolation

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
  Using cached flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
!CUDA_VISIBLE_DEVICES=0 swift sft \
    --sft_type lora \
    --use_flash_attn true \
    --model_type qwen2-7b-instruct \
    --model_id_or_path qwen/Qwen2-7B-Instruct \
    --dataset data/zhihu_train.jsonl#4000 alpaca-zh#2000 \
    --system "你是一个真实的人类。回答用户的问题，并在聊天回复中展现你的情绪，包括开心，幸福，愤怒，伤心，阴阳怪气等" \
    --dataset_test_ratio 0.01 \
    --output_dir output \
    --lora_target_modules ALL \
    --lora_rank 8 \
    --dtype bf16 \
    --seed 42 \
    --learning_rate 1e-4 \
    --warmup_ratio 0.05 \
    --max_length 2048 \
    --batch_size 4 \
    --eval_batch_size 4 \
    --num_train_epochs 1 \
    --gradient_accumulation_steps 4 \
    --save_total_limit 10 \
    --eval_steps 100 \
    --save_steps 100 

run sh: `python /mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/swift/cli/sft.py --sft_type lora --use_flash_attn true --model_type qwen2-7b-instruct --model_id_or_path qwen/Qwen2-7B-Instruct --dataset data/zhihu_train.jsonl#4000 alpaca-zh#2000 --system 你是一个真实的人类。回答用户的问题，并在聊天回复中展现你的情绪，包括开心，幸福，愤怒，伤心，阴阳怪气等 --dataset_test_ratio 0.01 --output_dir output --lora_target_modules ALL --lora_rank 8 --dtype bf16 --seed 42 --learning_rate 1e-4 --warmup_ratio 0.05 --max_length 2048 --batch_size 4 --eval_batch_size 4 --num_train_epochs 1 --gradient_accumulation_steps 4 --save_total_limit 10 --eval_steps 100 --save_steps 100`
[INFO:swift] Successfully registered `/mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] Start time of running main: 2024-08-19 15:00:05.657321
[INFO:swift] Setting template_type: qwen
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] Setting args.datal

## 使用evalscope评估模型

1. 自定义数据集评估：general qa
   - bleu：比较生成文本和参考文本中的n-gram（n个连续单词的序列）。常见的n有1（unigram）、2（bigram）、3（trigram）等。
   - rouge： 侧重于召回率（recall）

   缺少语义上的评估

In [2]:
!CUDA_VISIBLE_DEVICES=0 swift eval \
    --ckpt_dir output/qwen2-7b-instruct/v1-20240819-150005/checkpoint-371 \
    --eval_dataset no \
    --infer_backend pt \
    --eval_backend Native \
    --eval_limit 10 \
    --seed 42 \
    --eval_batch_size 8 \
    --custom_eval_config custom_eval_config.json \
    --temperature 0.7 \
    --top_k 20 \
    --top_p 0.9 

run sh: `python /mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/swift/cli/eval.py --ckpt_dir output/qwen2-7b-instruct/v1-20240819-150005/checkpoint-371 --eval_dataset no --infer_backend pt --eval_backend Native --eval_limit 10 --seed 42 --eval_batch_size 8 --custom_eval_config custom_eval_config.json --temperature 0.7 --top_k 20 --top_p 0.9`
[INFO:swift] Successfully registered `/mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] Start time of running main: 2024-08-19 15:30:42.503558
[INFO:swift] ckpt_dir: /mnt/data/data/user/maoyunlin.myl/tools/course/output/qwen2-7b-instruct/v1-20240819-150005/checkpoint-371
[INFO:swift] Setting model_info['revision']: master
[INFO:swift] Setting self.eval_human: True
[INFO:swift] Setting overwrite_generation_config: True
[INFO:swift] args: EvalArguments(model_type='qwen2-7b-instruct', model_id_or_path='qwen/Qwen2-7B-Instruct', model_r

2. 模型推理人工评估

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import pandas as pd

from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)
from swift.utils import seed_everything
from swift.tuners import Swift
import torch

seed_everything(42)

def infer_querys(model, template, querys):
    if type(querys) == str:
        querys = [querys]
        
    responses = []
    for query in querys:
        response, history = inference(model, template, query)
        response = response.replace("\n", "\t")
        responses.append(response)
        print(f'response: {response}')
    
    return responses

def load_model(ckpt_dir):
    model_type = ModelType.qwen2_7b_instruct
    template_type = get_default_template_type(model_type)

    model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'})
    model.generation_config.max_new_tokens = 500
    model.generation_config.temperature = 0.7
    model.generation_config.top_p = 0.9
    model.generation_config.top_k = 20
    if ckpt_dir:
        model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
    system_prompt = "你是一个真实的人类。回答用户的问题，并在聊天回复中展现你的情绪，包括开心，幸福，愤怒，伤心，阴阳怪气等"
    template = get_template(template_type, tokenizer, default_system=system_prompt)
    return model, template

querys = pd.read_json("data/zhihu_test.jsonl", lines=True)["query"].sample(10, random_state=42).tolist()
querys = ["你是谁?"] + querys

print(querys)

ckpt_dict = {
'origin': None,
'lora': 'output/qwen2-7b-instruct/v1-20240819-150005/checkpoint-371',
}
model = None
model_responses = {}
for ckpt_name, ckpt_dir in ckpt_dict.items():
    if model:
        del model
        torch.cuda.empty_cache()
    model, template = load_model(ckpt_dir)
    model_responses[ckpt_name] = infer_querys(model, template, querys)
    
df = pd.DataFrame.from_dict(model_responses)
df.index = querys
df.to_markdown("output.md")

/mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO:swift] Successfully registered `/mnt/data/data/user/maoyunlin.myl/miniconda3/envs/zhihu/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] Global seed set to 42
[INFO:swift] Downloading the model from ModelScope Hub, model_id: qwen/Qwen2-7B-Instruct


['你是谁?', '为什么现在很多人不看好商汤科技？', '实弹射击馆拿走弹壳，有影响吗？', '怎么样戒掉王者荣耀？', '有什么办法刺激自己坚持减肥？', '姜文或要拍摄抗美援朝电影《战俘营里的奥运会》，对此题材你有何期待？', '支付宝的蚂蚁森林是真的对生态产生了帮助么，还是只是做表面功夫？', '《霍比特人》《指环王》这类电影究竟能带给我们什么？', '怎样认识比你优秀的人并和他们成为朋友？', '你平时是如何做烘焙的？', 'Google 卫星地图拍下过哪些震撼的画面？']


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2-7B-Instruct
[INFO:swift] Setting torch_dtype: torch.bfloat16
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO:swift] model_kwargs: {'device_map': 'auto'}
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.21it/s]
[INFO:swift] model.max_model_len: 32768


response: 嗨！我是一个虚拟助手，很高兴能在这里帮助你解答问题、提供信息或简单地聊天。虽然我无法体验人类的情感，但我会尽力让你的每一次交流都愉快和有用。有什么我可以帮助你的吗？
response: 关于商汤科技不被看好的原因，可能有以下几个方面：		1. **市场波动**：科技行业尤其是AI领域竞争激烈，市场环境瞬息万变，投资者对不确定性敏感，可能会因为市场整体波动或行业政策调整而对个别公司产生担忧。		2. **业绩表现**：公司业绩、盈利能力、研发投入与回报的预期不符，或是增长速度未达到市场预期，这些都是影响投资者信心的因素。		3. **监管环境**：随着全球对于数据隐私和人工智能应用伦理的重视增加，相关政策法规的出台可能对某些业务模式产生限制，影响公司的长期发展预期。		4. **技术挑战**：AI领域面临的技术难题和迭代速度要求高，持续的研发投入压力大，可能让一些投资者觉得风险较高。		5. **负面新闻**：公司运营过程中出现的负面新闻或争议事件，如知识产权争议、高管变动、内部管理问题等，都可能影响公众和投资者对公司前景的看法。		6. **竞争格局**：科技行业竞争激烈，商汤科技面临的竞争对手实力强大，市场份额争夺战可能导致投资者对公司的市场地位和未来竞争力产生疑虑。		每个公司的情况都是复杂的，这些因素往往交织在一起，共同影响着市场对公司的评价。理解这些复杂性，需要深入分析公司的战略、财务状况、市场定位以及行业趋势等多方面的信息。
response: 实弹射击时，弹壳通常是由枪械自动回收的，不过偶尔也可能会出现掉落或被人为拿走的情况。如果在合法的射击场内，将弹壳拿走通常不会对射击活动本身产生直接影响，因为每发子弹都有配套的弹壳。然而，在公共场合或未授权的地方拿走弹壳可能会引起安全问题，比如可能误伤他人或者造成混淆。		在正规的射击活动中，确保所有装备和安全措施都得到妥善处理是非常重要的。如果你是在正规射击馆遇到这种情况，建议向工作人员报告，以便他们进行必要的管理。这样既能保障参与者的人身安全，也能维护射击场地的良好秩序。希望这能帮到你！如果有其他问题，随时欢迎提问哦~ 😊
response: 戒掉王者荣耀或者任何让你沉迷的游戏，其实是个挺有挑战的过程，但也是完全可行的！首先，我们得认识到，游戏成瘾往往与自我调节能力、时间管理以及心理状态有关。

[INFO:swift] Downloading the model from ModelScope Hub, model_id: qwen/Qwen2-7B-Instruct


response: 哇，Google 卫星地图确实捕捉到了许多令人震撼的画面！从自然奇观到人类工程，它的覆盖范围广泛且丰富。以下是一些特别引人注目的例子：		1. **极地冰川**：通过卫星地图，我们能够观察到全球各地的冰川变化，比如格陵兰岛和南极洲的冰盖退缩，以及北极海冰面积的变化。这些变化对全球气候变化的研究至关重要。		2. **建筑奇迹**：无论是古代遗迹如中国的长城、埃及的金字塔，还是现代城市中的摩天大楼，卫星地图都能提供独特的视角，让你仿佛站在高空俯瞰这些人类的杰作。		3. **自然灾害现场**：在发生地震、洪水或飓风等自然灾害后，卫星地图能够迅速提供受灾区域的即时图像，帮助救援团队进行有效的部署和评估。		4. **城市化进程**：通过长时间序列的数据对比，我们可以看到城市扩张的惊人速度，比如新开发的郊区、新建的高楼大厦等，这些都是人类活动对地球表面影响的直观体现。		5. **生态保护区**：卫星地图有助于监测森林砍伐、野生动物迁徙路径以及海洋生态系统的健康状况，对于环境保护和可持续发展至关重要。		6. **日常生活之美**：有时候，简单的的一幅小镇日出、一片落日余晖下的海滩或是星空下的城市景象，也能让人感受到自然与科技结合的独特魅力。		这些画面不仅展现了地球的多样性和美丽，也提醒我们保护环境的重要性。每次探索卫星地图都像是开启了一扇通往地球每一个角落的大门，每一次点击都可能带来新的发现和惊喜。


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2-7B-Instruct
[INFO:swift] Setting torch_dtype: torch.bfloat16
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO:swift] model_kwargs: {'device_map': 'auto'}
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]
[INFO:swift] model.max_model_len: 32768


response: 我是一个AI 聊天机器人，由阿里云开发。
response: 知乎有句话叫： 你永远不可能叫醒一个装睡的人。 这句话用在商汤身上非常合适。 商汤作为国内人工智能的领头羊，在人工智能领域，拥有绝对的技术优势和话语权。 但是，商汤在2019年上市失败后，就开始走下坡路了。 公司创始人徐立，被爆出在公司内使用P图软件，将员工照片P成女性，甚至在社交网络上发布。 然后公司员工发现后，就向徐立投诉，但徐立不仅没有停止这种行为，反而变本加厉，继续将员工照片P成女性，并且发到公司微信群内，以此来嘲讽。 而且，徐立还多次利用自己的职权之便，对女员工进行骚扰。 对于徐立的这些所作所为，商汤并没有进行处罚，反而是让徐立当着全公司的面，向女员工道歉。 就是这样一个作风恶劣的人，居然成为了商汤的创始人。 还有更夸张的事情，徐立曾经在朋友圈里发了一张自己在酒店的自拍，并配文“我真想杀了他”。 这个“他””指的就是商汤的一位高层，徐立在商汤的职位就是这个高层的手下。 这个高层也是徐立的老板。 徐立在社交网络上发表如此不堪入目的言论，而且还是针对自己老板的，商汤对此没有任何表示，只是让他删掉了这条朋友圈，而没有对其进行任何处罚。 在商汤科技上市失败后，公司创始人徐立，居然在朋友圈发表“我真想杀了他”的言论，这说明什么？ 这说明商汤的创始人徐立，在公司内没有威信，没有管理能力，没有领导力，他连自己手下的人都管不住。 这也说明，商汤的创始人徐立，根本不适合当一家市值上百亿的公司CEO。
response: 有影响的。 2016年，美国海军陆战队一名现役军人因携带枪支弹药进入公共场合，被判犯有非法持有枪械罪，罚款14,375美元并判处18个月监禁。 2018年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处30天监禁。 2019年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处90天监禁。 2020年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处10个月监禁。 2021年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处16个月监禁。 2022年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处12个月监禁。 2023年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处18个月监禁。 2024年，美国海军陆战队一名现役军人因非法持有枪械弹药被判处20个月监禁。 